# NORMAL DATAFRAME

In [1]:
from _lumber_prices import get_lumber_db_xl
from _lumber_schema import *
from adjusted_future import *

from datetime import datetime
import pandas as pd
import numpy as np

### Main Functions

In [2]:
def add_all_days_to_expiry(Lumber_expiry,total_idx,current_contract,current_contract_expiration):
    Lumber_expiry =pd.DataFrame(data= pd.concat([Lumber_expiry],axis=1), index = total_idx)
    Lumber_expiry.fillna(method='ffill',inplace = True)
    Lumber_expiry['symbol'].fillna(current_contract,inplace = True)
    Lumber_expiry['expirationDate'].fillna(current_contract_expiration,inplace = True)
    Lumber_expiry['expirationDate'] = pd.DatetimeIndex(Lumber_expiry['expirationDate'])
    return Lumber_expiry

def get_days_to_expiry(dates_dt, ctrct_expiry):
    from _helpers import cme
    from pandas.tseries.offsets import CustomBusinessDay
    us_bd = CustomBusinessDay(calendar=cme)
    days_to_expiry = len(pd.date_range(start=str(dates_dt),end=str(ctrct_expiry),freq=us_bd))
    return days_to_expiry

def get_expiry_dfs(month_seq,total_idx):
    Lumber_expiry, current_contract, current_contract_expiration, next_contract, next_contract_expiration = get_full_expiry_df(month_seq)
    Lumber_expiry.drop('Month_Start', axis=1, inplace=True)
    Lumber_expiry.drop('Expiry', axis=1, inplace=True)
    Lumber_expiry_shifted = Lumber_expiry
    Lumber_expiry_shifted = Lumber_expiry_shifted[Lumber_expiry_shifted.index <= next_contract_expiration]
    Lumber_expiry_shifted = Lumber_expiry_shifted.shift(month_seq-1).dropna()
    
    Lumber_expiry = add_all_days_to_expiry(Lumber_expiry,total_idx,current_contract,current_contract_expiration).sort_index(ascending=False)
    Lumber_expiry_shifted = add_all_days_to_expiry(Lumber_expiry_shifted,total_idx,next_contract,next_contract_expiration)

    for idx in Lumber_expiry.index:
        dates_dt = str(idx.strftime('%Y-%m-%d'))
        try:
            ctrct_expiry_shift = str(Lumber_expiry_shifted.loc[idx]['expirationDate'].strftime('%Y-%m-%d'))
            Lumber_expiry_shifted.loc[idx,'Days_to_expiry'] = get_days_to_expiry(dates_dt, ctrct_expiry_shift) # 1 means expiring today
        except:
            pass

    return Lumber_expiry_shifted

def get_full_expiry_df(month_seq):
        expiry_nearest = get_expiry()[['symbol','expirationDate']]
        expiry_nearest = expiry_nearest.dropna()
        Lumber_expiry = expiry_nearest[expiry_nearest['symbol'].str.startswith('LB')]
        Lumber_expiry.index = pd.DatetimeIndex(Lumber_expiry.expirationDate)
        # Lumber_expiry.drop('expirationDate', axis=1, inplace=True)
        Lumber_expiry.sort_index(ascending=False, inplace=True)
        Lumber_expiry['Month_Start'] = Lumber_expiry.index.strftime('%Y-%m-01')
        Lumber_expiry['Expiry'] = Lumber_expiry.index
        current_contract = Lumber_expiry[Lumber_expiry.index > datetime.now().strftime('%Y-%m-%d')].tail(1)['symbol'][0]
        current_contract_expiration = Lumber_expiry[Lumber_expiry.index > datetime.now().strftime('%Y-%m-%d')].tail(1)['Expiry'][0]
        next_contract = Lumber_expiry[Lumber_expiry.index > datetime.now().strftime('%Y-%m-%d')].tail(month_seq)['symbol'][0]
        next_contract_expiration = Lumber_expiry[Lumber_expiry.index > datetime.now().strftime('%Y-%m-%d')].tail(month_seq)['Expiry'][0]
   
        return Lumber_expiry, current_contract, current_contract_expiration, next_contract, next_contract_expiration


In [3]:
# ctrct_sequence = 1

def get_ctrct_mnth_df(ctrct_sequence):
    f = 'C:/source/2x4-data/app/data/Barchart/DailyDataDumpCME&NYMEX/Lumber_daily_price.csv'
    lumber_barchart = pd.DataFrame(pd.read_csv(f))
    lumber_barchart.index = pd.DatetimeIndex(lumber_barchart.tradingDay)
    lumber_barchart.index.name = 'date'
    lumber_barchart = lumber_barchart[lumber_barchart.Month_Seq == ctrct_sequence]
    lumber_barchart.drop_duplicates(subset='tradingDay', keep='first', inplace=True)
    lumber_barchart.sort_index(ascending=False, inplace=True)

    # Live Close Price from CWP Database
    Lumber_LIVE = get_lumber_db_xl() 
    Lumber = Lumber_LIVE.copy()

    # Create an index of all the dates in the Lumber_LIVE database as well Barchart for historical
    total_idx = pd.DatetimeIndex(pd.to_datetime(Lumber[ctrct_sequence].index.tolist() + lumber_barchart.index.tolist())).drop_duplicates(keep='first').sort_values(ascending=False)

    # Drpped Open because for now CWP db doesn't have Open Prices
    ohlcv_cols = ['high','low','close']

    Lumber[ctrct_sequence].index = pd.DatetimeIndex(Lumber[ctrct_sequence].index)

    Lumber[ctrct_sequence] = Lumber[ctrct_sequence].reindex(total_idx)
    lumber_barchart = lumber_barchart.reindex(total_idx)

    lumber_unadj = pd.concat([Lumber[ctrct_sequence][ohlcv_cols].dropna(how='all'),lumber_barchart[ohlcv_cols].dropna(how='all')])
    lumber_unadj = lumber_unadj[~lumber_unadj.index.duplicated(keep='first')]
    lumber_unadj.sort_index(ascending=False, inplace=True)

    # Chnage name of open interest column in Lumber_LIVE to match that in Barchart
    Lumber[ctrct_sequence] = Lumber[ctrct_sequence].rename(columns = {'open_interest':'openInterest'})

    # Combine Volume & OpenInterest from Barchart Historical Data (only available since about 2000-12-29) to live data from CWP Database - Shifting volume lower since Live CWP database captures volume & open interest for a day earlier
    lumber_volume_unadj = pd.concat([pd.DataFrame(Lumber[ctrct_sequence][['volume','openInterest']].shift(1)),pd.DataFrame(lumber_barchart[['volume','openInterest']])])
    lumber_volume_unadj = lumber_volume_unadj[~lumber_volume_unadj.index.duplicated(keep='first')]
    lumber_volume_unadj = lumber_volume_unadj[(lumber_volume_unadj !=0).all(1)]  # remove rows with 0 values and NANs
    lumber_volume_unadj.sort_index(ascending=False, inplace=True)

    # Combining OHLCV and Volume Data
    lumber_volume_unadj = lumber_volume_unadj.reindex(total_idx)
    lumber_unadj = pd.concat([lumber_unadj, lumber_volume_unadj], axis=1)
    lumber_unadj.sort_index(ascending=False, inplace=True)

    # Getting Contract symbol, Contract Expiry date & days_to_expiry by dates & then combining with lumber_unadj
    Lumber_expiry = get_expiry_dfs(month_seq=ctrct_sequence, total_idx=total_idx)
    lumber_unadj = pd.concat([lumber_unadj,Lumber_expiry], axis=1)

    # Lumber Cash Price
    cash_price = get_lumber_cash(code =['LCBQ', 'MAGQ'])
    cash_price = pd.DataFrame(cash_price.settle_phys)
    cash_price.columns = ['cash_price']
    cash_price.index.name = 'date'
    cash_price = cash_price.reindex(total_idx)
    lumber_unadj = pd.concat([lumber_unadj,cash_price], axis=1)
        
    lumber_unadj.columns = ['high','low','close','volume','openInterest','symbol','expirationDate','Days_to_expiry','cash_price']
    lumber_unadj = lumber_unadj[~lumber_unadj.index.duplicated(keep='first')] # removing duplicate indices
    lumber_unadj.sort_index(ascending=False, inplace=True)
    lumber_unadj.dropna(how ='all' , inplace=True)

    return lumber_unadj

In [4]:
# Lumber_LIVE = get_lumber_db_xl() 
# Lumber = Lumber_LIVE.copy()
# Lumber[1][300:]

In [5]:
# Takes 2 minutes to run
Lum_1 = get_ctrct_mnth_df(ctrct_sequence=1)
Lum_2 = get_ctrct_mnth_df(ctrct_sequence=2)

Getting data from CWP database & excel files::  33%|███▎      | 1/3 [00:04<00:08,  4.25s/it]

Time Taken for Contract:1 00:04


Getting data from CWP database & excel files::  67%|██████▋   | 2/3 [00:07<00:03,  3.40s/it]

Time Taken for Contract:2 00:07


Getting data from CWP database & excel files:: 100%|██████████| 3/3 [00:09<00:00,  3.28s/it]

...

Time Taken for Contract:1 00:02


Getting data from CWP database & excel files::  67%|██████▋   | 2/3 [00:05<00:02,  2.85s/it]

Time Taken for Contract:2 00:05


Getting data from CWP database & excel files:: 100%|██████████| 3/3 [00:08<00:00,  2.85s/it]

Time Taken for Contract:3 00:08


In [6]:
Lum_1[Lum_1.cash_price.notnull()]

,high,low,close,volume,openInterest,symbol,expirationDate,Days_to_expiry,cash_price
2022-07-27,561.599976,542.000000,555.200012,NaN,NaN,LBU22,2022-09-15,37.0,660.0
2022-07-26,574.500000,531.200012,556.599976,374.0,1446.0,LBU22,2022-09-15,38.0,660.0
2022-07-25,583.000000,554.299988,572.200012,280.0,1420.0,LBU22,2022-09-15,39.0,670.0
2022-07-22,617.000000,576.200012,585.299988,256.0,1354.0,LBU22,2022-09-15,40.0,670.0
2022-07-21,630.000000,599.400024,623.200012,243.0,1379.0,LBU22,2022-09-15,41.0,670.0
...,...,...,...,...,...,...,...,...,...
2006-09-29,245.200000,240.600000,240.600000,NaN,NaN,LBX06,2006-11-15,34.0,247.1
2006-09-22,259.000000,256.200000,258.500000,NaN,NaN,LBX06,2006-11-15,39.0,262.2
2006-09-15,273.000000,262.000000,262.000000,NaN,NaN,LBU06,2006-09-15,1.0,267.1
2006-09-08,280.000000,273.700000,279.700000,NaN,NaN,LBU06,2006-09-15,6.0,290.7


In [7]:
Lum_2[:20]

,high,low,close,volume,openInterest,symbol,expirationDate,Days_to_expiry,cash_price
2022-07-27,565.299988,555.000000,560.000000,NaN,NaN,LBX22,2022-11-15,80.0,660.0
2022-07-26,578.000000,545.099976,562.599976,58.0,397.0,LBX22,2022-11-15,81.0,660.0
2022-07-25,581.500000,563.599976,581.500000,39.0,386.0,LBX22,2022-11-15,82.0,670.0
2022-07-22,606.500000,580.799988,586.000000,43.0,392.0,LBX22,2022-11-15,83.0,670.0
2022-07-21,622.099976,598.200012,622.099976,32.0,397.0,LBX22,2022-11-15,84.0,670.0
2022-07-20,NaN,NaN,630.200012,4.0,396.0,LBX22,2022-11-15,85.0,670.0
2022-07-19,640.599976,622.599976,634.900024,47.0,396.0,LBX22,2022-11-15,86.0,670.0
2022-07-18,647.500000,NaN,642.599976,24.0,373.0,LBX22,2022-11-15,87.0,670.0
2022-07-15,650.000000,618.099976,648.799988,15.0,362.0,LBU22,2022-09-15,45.0,670.0
2022-07-14,664.200012,634.299988,646.000000,277.0,1449.0,LBU22,2022-09-15,46.0,670.0


### Switching Logics

In [8]:
# 1. Never Switching - We still need to use previous close price on day of expiry since we need to calculate adj return when new contract starts 
# In this case just shifting list to the left - the last index won't matter as it is
switch_list_1 = [item for sublist in np.where([Lum_1.index == Lum_1.expirationDate],1,0).tolist() for item in sublist]
switch_list_1.append(switch_list_1.pop(0))
len(switch_list_1)

13559

In [9]:
# 2. Switching on the day of expiry
switch_list_2 = [item for sublist in np.where([Lum_1.index == Lum_1.expirationDate],1,0).tolist() for item in sublist]
len(switch_list_2)

13559

In [10]:
# 3. Switch to 2nd month at tne start of contract expiry month
dt_lists = []
for end_dt in Lum_1.expirationDate.unique():
    end_dt = pd.to_datetime(end_dt)
    for dt in pd.date_range(start=end_dt.strftime("%Y-%m-1"), end=end_dt):
        if dt in Lum_1.index:
            dt_lists.append(dt)

switch_list_3 = [1 if dt in dt_lists else 0 for dt in Lum_1.index]
len(switch_list_3)

13559

In [11]:
(Lum_2[Lum_2.cash_price.notnull()]['cash_price'] == Lum_1[Lum_1.cash_price.notnull()]['cash_price']).all()

True

### Calculates Adjusted Returns, Basis and Basis by DaysToExpiry based on Switching Lists

In [12]:
def calculate_adj_ret(Lum_1,Lum_2, switch_list, switch_second):
    Adj_Lum = Lum_1.copy(deep=True)
    cols_to_replace = ['symbol', 'expirationDate', 'Days_to_expiry','close','cash_price']
    Adj_Lum = Lum_1[cols_to_replace]
    Adj_Lum['Switch'] = switch_list

    # Fill rows with second contract where switch is 1 when switch_second equals True - This column is used to calculate returns for same contract
    if switch_second == True:
        for dt in Adj_Lum.index:
            if (Adj_Lum.loc[dt, 'Switch'] == 1) & (dt in Lum_2.index):
                for col in cols_to_replace:
                    Adj_Lum.loc[dt, col] = Lum_2.loc[dt, col]

    # Fill Next Close column (previous date) with 2nd Contract's Close Price (previous date) wherever switch is 1
    for l,dt in zip(range(len(Adj_Lum)),Adj_Lum.index):
        try:
            idx = Adj_Lum.index.get_loc(Adj_Lum[Adj_Lum.Switch == 1].index[l])
            dt_to_replace = Adj_Lum.index[idx+1]
            Adj_Lum.loc[dt_to_replace, 'Next_Close'] = Lum_2.loc[dt_to_replace, 'close']
            
        except:
            pass

    # Calculate Adj Returns - Where switch is 0, calculate simple log returns based on only 'Close' column but when Switch is 1, calculate log returns based on 'Close' and 'Next_Close' columns
    for l,dt in zip(range(len(Adj_Lum)),Adj_Lum.index):
        try:
            if (Adj_Lum.loc[dt, 'Switch'] == 1):
                Adj_Lum.loc[dt, 'Adj_ret'] = np.log(Adj_Lum.iloc[l]['close']/Adj_Lum.iloc[l+1]['Next_Close'])
            else:
                Adj_Lum.loc[dt, 'Adj_ret'] = np.log(Adj_Lum.iloc[l]['close']/Adj_Lum.iloc[l+1]['close'])
            
        except:
            pass
    
    Adj_Lum['Basis'] = Adj_Lum['cash_price'] - Adj_Lum['close']
    Adj_Lum['BasisByDte'] = Adj_Lum['Basis'] / Adj_Lum['Days_to_expiry']

    Adj_Lum.drop(['Switch','Next_Close'], axis=1, inplace=True)

    return Adj_Lum
    

In [13]:
# Takes 1 minute to run
Adj_Lum_1 = calculate_adj_ret(Lum_1,Lum_2, switch_list_1,switch_second=False)
Adj_Lum_2 = calculate_adj_ret(Lum_1,Lum_2, switch_list_2,switch_second=True)
Adj_Lum_3 = calculate_adj_ret(Lum_1,Lum_2, switch_list_3,switch_second=True)

In [42]:
Adj_Lum_1

,symbol,expirationDate,Days_to_expiry,close,cash_price,Adj_ret,Basis,BasisByDte
2022-07-27,LBU22,2022-09-15,37.0,555.200012,660.0,-0.002518,104.799988,2.832432
2022-07-26,LBU22,2022-09-15,38.0,556.599976,660.0,-0.027642,103.400024,2.721053
2022-07-25,LBU22,2022-09-15,39.0,572.200012,670.0,-0.022636,97.799988,2.507692
2022-07-22,LBU22,2022-09-15,40.0,585.299988,670.0,-0.062743,84.700012,2.117500
2022-07-21,LBU22,2022-09-15,41.0,623.200012,670.0,-0.013389,46.799988,1.141463
...,...,...,...,...,...,...,...,...
1969-10-07,LBH70,1970-03-13,114.0,79.000000,NaN,-0.012579,NaN,NaN
1969-10-06,LBH70,1970-03-13,115.0,80.000000,NaN,0.000000,NaN,NaN
1969-10-03,LBH70,1970-03-13,116.0,80.000000,NaN,-0.006231,NaN,NaN
1969-10-02,LBH70,1970-03-13,117.0,80.500000,NaN,0.006231,NaN,NaN


In [40]:
cols_to_insert = ['close','Adj_ret','Basis','BasisByDte','Days_to_expiry']
df1 = Adj_Lum_1[cols_to_insert]
df1.columns = ['Close_1','Adj_ret_1','Basis_1','BasisByDte_1','DTE_1']
df2 = Adj_Lum_2[cols_to_insert]
df2.columns = ['Close_2','Adj_ret_2','Basis_2','BasisByDte_2','DTE_2']
df3 = Adj_Lum_3[cols_to_insert]
df3.columns = ['Close_3','Adj_ret_3','Basis_3','BasisByDte_3','DTE_3']
df = pd.concat([df1,df2,df3], axis=1)
df.head(2).append(df.tail(2))

,Close_1,Adj_ret_1,Basis_1,BasisByDte_1,DTE_1,Close_2,Adj_ret_2,Basis_2,BasisByDte_2,DTE_2,Close_3,Adj_ret_3,Basis_3,BasisByDte_3,DTE_3
2022-07-27,555.200012,-0.002518,104.799988,2.832432,37.0,555.200012,-0.002518,104.799988,2.832432,37.0,555.200012,-0.002518,104.799988,2.832432,37.0
2022-07-26,556.599976,-0.027642,103.400024,2.721053,38.0,556.599976,-0.027642,103.400024,2.721053,38.0,556.599976,-0.027642,103.400024,2.721053,38.0
2022-07-25,572.200012,-0.022636,97.799988,2.507692,39.0,572.200012,-0.022636,97.799988,2.507692,39.0,572.200012,-0.022636,97.799988,2.507692,39.0
2022-07-22,585.299988,-0.062743,84.700012,2.117500,40.0,585.299988,-0.062743,84.700012,2.117500,40.0,585.299988,-0.062743,84.700012,2.117500,40.0
2022-07-21,623.200012,-0.013389,46.799988,1.141463,41.0,623.200012,-0.013389,46.799988,1.141463,41.0,623.200012,-0.013389,46.799988,1.141463,41.0
1969-10-07,79.000000,-0.012579,NaN,NaN,114.0,79.000000,-0.012579,NaN,NaN,114.0,79.000000,-0.012579,NaN,NaN,114.0
1969-10-06,80.000000,0.000000,NaN,NaN,115.0,80.000000,0.000000,NaN,NaN,115.0,80.000000,0.000000,NaN,NaN,115.0
1969-10-03,80.000000,-0.006231,NaN,NaN,116.0,80.000000,-0.006231,NaN,NaN,116.0,80.000000,-0.006231,NaN,NaN,116.0
1969-10-02,80.500000,0.006231,NaN,NaN,117.0,80.500000,0.006231,NaN,NaN,117.0,80.500000,0.006231,NaN,NaN,117.0
1969-10-01,80.000000,NaN,NaN,NaN,118.0,80.000000,NaN,NaN,NaN,118.0,80.000000,NaN,NaN,NaN,118.0


### What has been the basis - both range and average on the Days of Expiry - Using Adj_Lum_1 since that is the only one wITH Adj_Lum_1.index == Adj_Lum_1.expirationDate

In [14]:
pd.DataFrame(Adj_Lum_1[Adj_Lum_1.index == Adj_Lum_1.expirationDate]['Basis']).describe()

,Basis
count,63.000000
mean,5.111112
std,29.423604
min,-102.000000
25%,-9.199988
50%,3.000000
75%,14.500000
max,135.000000


In [15]:
Adj_Lum_1.BasisByDte.corr(Adj_Lum_1.Adj_ret.shift(-1)), Adj_Lum_2.BasisByDte.corr(Adj_Lum_2.Adj_ret.shift(-1)), Adj_Lum_3.BasisByDte.corr(Adj_Lum_3.Adj_ret.shift(-1))

(-0.1769677350313087, -0.24708197133037682, -0.26951136722808566)

In [16]:
Adj_Lum_2[:20]

,symbol,expirationDate,Days_to_expiry,close,cash_price,Adj_ret,Basis,BasisByDte
2022-07-27,LBU22,2022-09-15,37.0,555.200012,660.0,-0.002518,104.799988,2.832432
2022-07-26,LBU22,2022-09-15,38.0,556.599976,660.0,-0.027642,103.400024,2.721053
2022-07-25,LBU22,2022-09-15,39.0,572.200012,670.0,-0.022636,97.799988,2.507692
2022-07-22,LBU22,2022-09-15,40.0,585.299988,670.0,-0.062743,84.700012,2.117500
2022-07-21,LBU22,2022-09-15,41.0,623.200012,670.0,-0.013389,46.799988,1.141463
2022-07-20,LBU22,2022-09-15,42.0,631.599976,670.0,-0.023008,38.400024,0.914286
2022-07-19,LBU22,2022-09-15,43.0,646.299988,670.0,0.008546,23.700012,0.551163
2022-07-18,LBU22,2022-09-15,44.0,640.799988,670.0,-0.012407,29.200012,0.663637
2022-07-15,LBU22,2022-09-15,45.0,648.799988,670.0,0.004325,21.200012,0.471111
2022-07-14,LBN22,2022-07-15,2.0,640.000000,670.0,-0.018576,30.000000,15.000000


In [17]:
pd.DataFrame(Adj_Lum_1[Adj_Lum_1.index == Adj_Lum_1.expirationDate]['Basis']).describe()

,Basis
count,63.000000
mean,5.111112
std,29.423604
min,-102.000000
25%,-9.199988
50%,3.000000
75%,14.500000
max,135.000000


In [18]:
Adj_Lum_2[6:]

,symbol,expirationDate,Days_to_expiry,close,cash_price,Adj_ret,Basis,BasisByDte
2022-07-19,LBU22,2022-09-15,43.0,646.299988,670.0,0.008546,23.700012,0.551163
2022-07-18,LBU22,2022-09-15,44.0,640.799988,670.0,-0.012407,29.200012,0.663637
2022-07-15,LBU22,2022-09-15,45.0,648.799988,670.0,0.004325,21.200012,0.471111
2022-07-14,LBN22,2022-07-15,2.0,640.000000,670.0,-0.018576,30.000000,15.000000
2022-07-13,LBN22,2022-07-15,3.0,652.000000,660.0,0.010794,8.000000,2.666667
...,...,...,...,...,...,...,...,...
1969-10-07,LBH70,1970-03-13,114.0,79.000000,NaN,-0.012579,NaN,NaN
1969-10-06,LBH70,1970-03-13,115.0,80.000000,NaN,0.000000,NaN,NaN
1969-10-03,LBH70,1970-03-13,116.0,80.000000,NaN,-0.006231,NaN,NaN
1969-10-02,LBH70,1970-03-13,117.0,80.500000,NaN,0.006231,NaN,NaN


In [19]:
Adj_Lum_3.Adj_ret.max(), Adj_Lum_3.Adj_ret.min()

(0.10376256971308043, -0.09252264990408156)

In [20]:
Adj_Lum_3[6:20]

,symbol,expirationDate,Days_to_expiry,close,cash_price,Adj_ret,Basis,BasisByDte
2022-07-19,LBU22,2022-09-15,43.0,646.299988,670.0,0.008546,23.700012,0.551163
2022-07-18,LBU22,2022-09-15,44.0,640.799988,670.0,-0.012407,29.200012,0.663637
2022-07-15,LBU22,2022-09-15,45.0,648.799988,670.0,0.004325,21.200012,0.471111
2022-07-14,LBU22,2022-09-15,46.0,646.000000,670.0,-0.056134,24.000000,0.521739
2022-07-13,LBU22,2022-09-15,47.0,683.299988,660.0,0.005871,-23.299988,-0.495744
2022-07-12,LBU22,2022-09-15,48.0,679.299988,660.0,-0.025436,-19.299988,-0.402083
2022-07-11,LBU22,2022-09-15,49.0,696.799988,640.0,-0.014532,-56.799988,-1.159183
2022-07-08,LBU22,2022-09-15,50.0,707.000000,640.0,-0.026382,-67.000000,-1.340000
2022-07-07,LBU22,2022-09-15,51.0,725.900024,640.0,0.055806,-85.900024,-1.684314
2022-07-06,LBU22,2022-09-15,52.0,686.500000,630.0,0.057098,-56.500000,-1.086538


In [21]:
np.log(640.799988/648.799988)

-0.012407107277961073

In [22]:
Adj_Lum_1[:20]

,symbol,expirationDate,Days_to_expiry,close,cash_price,Adj_ret,Basis,BasisByDte
2022-07-27,LBU22,2022-09-15,37.0,555.200012,660.0,-0.002518,104.799988,2.832432
2022-07-26,LBU22,2022-09-15,38.0,556.599976,660.0,-0.027642,103.400024,2.721053
2022-07-25,LBU22,2022-09-15,39.0,572.200012,670.0,-0.022636,97.799988,2.507692
2022-07-22,LBU22,2022-09-15,40.0,585.299988,670.0,-0.062743,84.700012,2.117500
2022-07-21,LBU22,2022-09-15,41.0,623.200012,670.0,-0.013389,46.799988,1.141463
2022-07-20,LBU22,2022-09-15,42.0,631.599976,670.0,-0.023008,38.400024,0.914286
2022-07-19,LBU22,2022-09-15,43.0,646.299988,670.0,0.008546,23.700012,0.551163
2022-07-18,LBU22,2022-09-15,44.0,640.799988,670.0,-0.012407,29.200012,0.663637
2022-07-15,LBN22,2022-07-15,1.0,682.000000,670.0,0.063561,-12.000000,-12.000000
2022-07-14,LBN22,2022-07-15,2.0,640.000000,670.0,-0.018576,30.000000,15.000000


In [23]:
Lum_2[7:]

,high,low,close,volume,openInterest,symbol,expirationDate,Days_to_expiry,cash_price
2022-07-18,647.500000,NaN,642.599976,24.0,373.0,LBX22,2022-11-15,87.0,670.0
2022-07-15,650.000000,618.099976,648.799988,15.0,362.0,LBU22,2022-09-15,45.0,670.0
2022-07-14,664.200012,634.299988,646.000000,277.0,1449.0,LBU22,2022-09-15,46.0,670.0
2022-07-13,695.000000,670.200012,683.299988,121.0,1456.0,LBU22,2022-09-15,47.0,660.0
2022-07-12,700.000000,671.500000,679.299988,125.0,1489.0,LBU22,2022-09-15,48.0,660.0
...,...,...,...,...,...,...,...,...,...
1969-10-07,82.000000,81.500000,82.000000,NaN,NaN,LBK70,1970-05-15,159.0,NaN
1969-10-06,81.500000,81.500000,81.500000,NaN,NaN,LBK70,1970-05-15,160.0,NaN
1969-10-03,81.500000,81.500000,81.500000,NaN,NaN,LBK70,1970-05-15,161.0,NaN
1969-10-02,81.250000,81.000000,81.250000,NaN,NaN,LBK70,1970-05-15,162.0,NaN


In [24]:
Lum_1

,high,low,close,volume,openInterest,symbol,expirationDate,Days_to_expiry,cash_price
2022-07-27,561.599976,542.000000,555.200012,NaN,NaN,LBU22,2022-09-15,37.0,660.0
2022-07-26,574.500000,531.200012,556.599976,374.0,1446.0,LBU22,2022-09-15,38.0,660.0
2022-07-25,583.000000,554.299988,572.200012,280.0,1420.0,LBU22,2022-09-15,39.0,670.0
2022-07-22,617.000000,576.200012,585.299988,256.0,1354.0,LBU22,2022-09-15,40.0,670.0
2022-07-21,630.000000,599.400024,623.200012,243.0,1379.0,LBU22,2022-09-15,41.0,670.0
...,...,...,...,...,...,...,...,...,...
1969-10-07,79.000000,78.000000,79.000000,NaN,NaN,LBH70,1970-03-13,114.0,NaN
1969-10-06,80.500000,80.000000,80.000000,NaN,NaN,LBH70,1970-03-13,115.0,NaN
1969-10-03,80.000000,80.000000,80.000000,NaN,NaN,LBH70,1970-03-13,116.0,NaN
1969-10-02,80.500000,79.500000,80.500000,NaN,NaN,LBH70,1970-03-13,117.0,NaN


In [25]:
# Get workbook and worksheet objects       
# writer = pd.ExcelWriter('Adjust_prices.xlsx')  
# Adj_Lum_1.to_excel(writer, sheet_name='Lum_1')
# Adj_Lum_2.to_excel(writer, sheet_name='Lum_2')
# Adj_Lum_3.to_excel(writer, sheet_name='Lum_3')
# writer.close()

In [26]:
# writer = pd.ExcelWriter('Lums.xlsx')  
# Lum_1.to_excel(writer, sheet_name='Lum_1')
# Lum_2.to_excel(writer, sheet_name='Lum_2')
# writer.close()

In [27]:
Adj_Lum_2.Adj_ret.max(), Adj_Lum_2.Adj_ret.min()

(0.14194910150356696, -0.9404567977213387)

In [28]:
Adj_Lum_2.Adj_ret.argmax(), Adj_Lum_2.Adj_ret.argmin()

(10422, 10421)

In [29]:
(Adj_Lum_2.Adj_ret == Adj_Lum_1.Adj_ret)[6:]

2022-07-19     True
2022-07-18     True
2022-07-15    False
2022-07-14     True
2022-07-13     True
              ...  
1969-10-07     True
1969-10-06     True
1969-10-03     True
1969-10-02     True
1969-10-01    False
Name: Adj_ret, Length: 13553, dtype: bool

In [30]:
Adj_Lum_2.iloc[10421]

symbol                          LBZ82
expirationDate    1982-04-30 00:00:00
Days_to_expiry                   15.0
close                           62.55
cash_price                        NaN
Adj_ret                     -0.940457
Basis                             NaN
BasisByDte                        NaN
Name: 1982-04-12 00:00:00, dtype: object

In [31]:
Adj_Lum_1.Adj_ret.max(), Adj_Lum_1.Adj_ret.min()

(0.1419447741477546, -0.9404567977213387)

In [32]:
Adj_Lum_1.iloc[:Adj_Lum_1.Adj_ret.argmax()].max()['Adj_ret'], Adj_Lum_1.iloc[:Adj_Lum_1.Adj_ret.argmin()].min()['Adj_ret']

(0.10244531167267354, -0.1387440497997851)

In [33]:
Adj_Lum_1[Adj_Lum_1.close.notnull()]

,symbol,expirationDate,Days_to_expiry,close,cash_price,Adj_ret,Basis,BasisByDte
2022-07-27,LBU22,2022-09-15,37.0,555.200012,660.0,-0.002518,104.799988,2.832432
2022-07-26,LBU22,2022-09-15,38.0,556.599976,660.0,-0.027642,103.400024,2.721053
2022-07-25,LBU22,2022-09-15,39.0,572.200012,670.0,-0.022636,97.799988,2.507692
2022-07-22,LBU22,2022-09-15,40.0,585.299988,670.0,-0.062743,84.700012,2.117500
2022-07-21,LBU22,2022-09-15,41.0,623.200012,670.0,-0.013389,46.799988,1.141463
...,...,...,...,...,...,...,...,...
1969-10-07,LBH70,1970-03-13,114.0,79.000000,NaN,-0.012579,NaN,NaN
1969-10-06,LBH70,1970-03-13,115.0,80.000000,NaN,0.000000,NaN,NaN
1969-10-03,LBH70,1970-03-13,116.0,80.000000,NaN,-0.006231,NaN,NaN
1969-10-02,LBH70,1970-03-13,117.0,80.500000,NaN,0.006231,NaN,NaN


# You Don't need the following functions - Check

In [41]:
# Get Expiry dataframe from csv
def get_expiry():
    expiry_nearest = pd.read_csv("C:/source/2x4-data/app/data/Barchart/DailyDataDumpCME&NYMEX/expiry_df.csv", index_col=0)
    return expiry_nearest

In [34]:
from _lumber_prices import get_lumber_db_xl
from datetime import datetime
# from adjusted_future import *
import pandas as pd
import numpy as np
new_col_name = 'MnthStart_Adj_Close'


# Get Expiry dataframe from csv
def get_expiry():
    expiry_nearest = pd.read_csv("C:/source/2x4-data/app/data/Barchart/DailyDataDumpCME&NYMEX/expiry_df.csv", index_col=0)
    return expiry_nearest

def add_all_days_to_expiry(Lumber_expiry,Lumber_1,current_contract,current_contract_expiration):
    Lumber_expiry =pd.DataFrame(data= pd.concat([Lumber_expiry],axis=1), index = Lumber_1.index)
    Lumber_expiry.fillna(method='ffill',inplace = True)
    Lumber_expiry['symbol'].fillna(current_contract,inplace = True)
    Lumber_expiry['expirationDate'].fillna(current_contract_expiration,inplace = True)
    Lumber_expiry['expirationDate'] = pd.DatetimeIndex(Lumber_expiry['expirationDate'])
    return Lumber_expiry

def get_days_to_expiry(dates_dt, ctrct_expiry):
    from _helpers import cme
    from pandas.tseries.offsets import CustomBusinessDay
    us_bd = CustomBusinessDay(calendar=cme)
    days_to_expiry = len(pd.date_range(start=str(dates_dt),end=str(ctrct_expiry),freq=us_bd))
    return days_to_expiry

def get_LB_expiry():
    expiry_nearest = get_expiry()[['symbol','expirationDate']]
    expiry_nearest = expiry_nearest.dropna()
    Lumber_expiry = expiry_nearest[expiry_nearest['symbol'].str.startswith('LB')]
    Lumber_expiry.index = pd.DatetimeIndex(Lumber_expiry.expirationDate)
    # Lumber_expiry.drop('expirationDate', axis=1, inplace=True)
    Lumber_expiry.sort_index(ascending=False, inplace=True)
    Lumber_expiry['Month_Start'] = Lumber_expiry.index.strftime('%Y-%m-01')
    Lumber_expiry['Expiry'] = Lumber_expiry.index
    current_contract = Lumber_expiry[Lumber_expiry.index > datetime.now().strftime('%Y-%m-%d')].tail(1)['symbol'][0]
    current_contract_expiration = Lumber_expiry[Lumber_expiry.index > datetime.now().strftime('%Y-%m-%d')].tail(1)['Expiry'][0]
    Lumber_expiry = Lumber_expiry[Lumber_expiry.index < current_contract_expiration]
    return Lumber_expiry, current_contract, current_contract_expiration


def get_expiries_dfs(lumber_1):
    Lumber_expiry, current_contract, current_contract_expiration = get_LB_expiry()
    Lumber_expiry.drop('Month_Start', axis=1, inplace=True)
    Lumber_expiry.drop('Expiry', axis=1, inplace=True)
    Lumber_expiry_shifted = Lumber_expiry.shift(1)

    Lumber_expiry = add_all_days_to_expiry(Lumber_expiry,lumber_1,current_contract,current_contract_expiration)
    Lumber_expiry_shifted = add_all_days_to_expiry(Lumber_expiry_shifted,lumber_1,current_contract,current_contract_expiration)

    for idx in Lumber_expiry.index:
        dates_dt = str(idx.strftime('%Y-%m-%d'))
        ctrct_expiry = str(Lumber_expiry.loc[idx]['expirationDate'].strftime('%Y-%m-%d'))
        Lumber_expiry.loc[idx,'Days_to_expiry'] = get_days_to_expiry(dates_dt, ctrct_expiry) #- 1 # so 0 means expiring today

        try:
            ctrct_expiry_shift = str(Lumber_expiry_shifted.loc[idx]['expirationDate'].strftime('%Y-%m-%d'))
            Lumber_expiry_shifted.loc[idx,'Days_to_expiry'] = get_days_to_expiry(dates_dt, ctrct_expiry_shift) #- 1 # so 0 means expiring today
        except:
            pass
    return Lumber_expiry, Lumber_expiry_shifted


def get_adjusted_df():

    Lumber = get_lumber_db_xl() 
    Lumber_expiry, _ , _ = get_LB_expiry()
    Lumber_expiry_normal, Lumber_expiry_shifted = get_expiries_dfs(Lumber[1])
    Lumber_Adjusted_Prices_MnthStart = Lumber[1].copy(deep=True)
    
    Lumber_Adjusted_Prices_MnthStart[new_col_name] = np.log(Lumber_Adjusted_Prices_MnthStart.close ) - np.log(Lumber_Adjusted_Prices_MnthStart.close.shift(-1))            

    for expiry_dt in Lumber_expiry.index:
        try:
            for dt in pd.date_range(start=Lumber_expiry.loc[expiry_dt,'Month_Start'], end=Lumber_expiry.loc[expiry_dt,'Expiry']):
                dt = dt.strftime('%Y-%m-%d')
            
                if dt in Lumber_Adjusted_Prices_MnthStart.index:
                    Lumber_Adjusted_Prices_MnthStart.loc[dt] = Lumber[2].loc[dt]
                    # Lumber_Adjusted_Prices_MnthStart.loc[dt,Lumber_Adjusted_Prices_MnthStart.columns.drop(['Days_to_expiry','symbol'])] = Lumber[2].loc[dt]

        except:
            pass

    Lumber_Adjusted_Prices_MnthStart[new_col_name] = np.log(Lumber_Adjusted_Prices_MnthStart.close) - np.log(Lumber_Adjusted_Prices_MnthStart.close.shift(-1))

    for dt in Lumber_expiry.Month_Start.values:
        try:
            if (pd.to_datetime(dt) >= Lumber_Adjusted_Prices_MnthStart.index[-1]):
                dt_closest_above = (Lumber_Adjusted_Prices_MnthStart[Lumber_Adjusted_Prices_MnthStart.index >= dt].index[-1])
                dt_closest_below = (Lumber_Adjusted_Prices_MnthStart[Lumber_Adjusted_Prices_MnthStart.index < dt].index[0])
                Lumber_Adjusted_Prices_MnthStart.loc[dt_closest_above,new_col_name] = np.log(Lumber_Adjusted_Prices_MnthStart.loc[dt_closest_above,'close']) - np.log(Lumber[2].loc[dt_closest_below,'close'])
        except:
            pass

    Lumber_Adjusted_Prices_MnthStart['Days_to_expiry'] = Lumber_expiry_normal['Days_to_expiry']
    Lumber_Adjusted_Prices_MnthStart['symbol'] = Lumber_expiry_normal['symbol']

    Lumber_Adjusted_Prices_MnthStart['Days_to_expiry_adj'] = Lumber_expiry_normal['Days_to_expiry']
    Lumber_Adjusted_Prices_MnthStart['symbol_adj'] = Lumber_expiry_normal['symbol']

    # For now Using Days to expiry as original but if you want it to match Adjusted Close, uncomment the code below
    for expiry_dt in Lumber_expiry.index:
        try:
            for dt in pd.date_range(start=Lumber_expiry.loc[expiry_dt,'Month_Start'], end=Lumber_expiry.loc[expiry_dt,'Expiry']):
                dt = dt.strftime('%Y-%m-%d')
            
                if dt in Lumber_Adjusted_Prices_MnthStart.index:
                    Lumber_Adjusted_Prices_MnthStart.loc[dt,'Days_to_expiry_adj'] = Lumber_expiry_shifted.loc[dt,'Days_to_expiry']
                    Lumber_Adjusted_Prices_MnthStart.loc[dt,'symbol_adj'] = Lumber_expiry_shifted.loc[dt,'symbol']
        except:
            pass
    
    return Lumber_Adjusted_Prices_MnthStart

Lumber_Adjusted_Prices_MnthStart = get_adjusted_df()
Lumber_Adjusted_Prices_MnthStart.head().append(Lumber_Adjusted_Prices_MnthStart.tail())

Getting data from CWP database & excel files::  33%|███▎      | 1/3 [00:02<00:05,  2.99s/it]

Time Taken for Contract:1 00:02


Getting data from CWP database & excel files::  67%|██████▋   | 2/3 [00:05<00:02,  2.93s/it]

Time Taken for Contract:2 00:05


Getting data from CWP database & excel files:: 100%|██████████| 3/3 [00:08<00:00,  2.97s/it]

Time Taken for Contract:3 00:08


,open,high,low,close,volume,open_interest,MnthStart_Adj_Close,Days_to_expiry,symbol,Days_to_expiry_adj,symbol_adj
date,,,,,,,,,,,
2022-07-27,None,561.599976,542.000000,555.200012,374.0,1446.0,-0.002518,37.0,LBU22,37.0,LBU22
2022-07-26,None,574.500000,531.200012,556.599976,280.0,1420.0,-0.027642,38.0,LBU22,38.0,LBU22
2022-07-25,None,583.000000,554.299988,572.200012,256.0,1354.0,-0.022636,39.0,LBU22,39.0,LBU22
2022-07-22,None,617.000000,576.200012,585.299988,243.0,1379.0,-0.062743,40.0,LBU22,40.0,LBU22
2022-07-21,None,630.000000,599.400024,623.200012,127.0,1406.0,-0.013389,41.0,LBU22,41.0,LBU22
2011-01-13,308.0,309.500000,303.200000,304.200000,291.0,6956.0,0.004283,2.0,LBF11,44.0,LBH11
2011-01-12,302.6,309.400000,298.800000,302.900000,477.0,6964.0,0.000330,3.0,LBF11,45.0,LBH11
2011-01-11,310.8,310.800000,302.800000,302.800000,1035.0,6857.0,-0.032491,4.0,LBF11,46.0,LBH11
2011-01-10,324.1,324.100000,312.800000,312.800000,737.0,7268.0,-0.031469,5.0,LBF11,47.0,LBH11


In [35]:

# total_idx = pd.DatetimeIndex(pd.to_datetime(Lumber[ctrct_sequence].index.tolist() + lumber_barchart.index.tolist())).drop_duplicates(keep='first').sort_values(ascending=False)



def get_LB_expiry():
    expiry_nearest = get_expiry()[['symbol','expirationDate']]
    expiry_nearest = expiry_nearest.dropna()
    Lumber_expiry = expiry_nearest[expiry_nearest['symbol'].str.startswith('LB')]
    Lumber_expiry.index = pd.DatetimeIndex(Lumber_expiry.expirationDate)
    # Lumber_expiry.drop('expirationDate', axis=1, inplace=True)
    Lumber_expiry.sort_index(ascending=False, inplace=True)
    Lumber_expiry['Month_Start'] = Lumber_expiry.index.strftime('%Y-%m-01')
    Lumber_expiry['Expiry'] = Lumber_expiry.index
    current_contract = Lumber_expiry[Lumber_expiry.index > datetime.now().strftime('%Y-%m-%d')].tail(1)['symbol'][0]
    current_contract_expiration = Lumber_expiry[Lumber_expiry.index > datetime.now().strftime('%Y-%m-%d')].tail(1)['Expiry'][0]
    Lumber_expiry = Lumber_expiry[Lumber_expiry.index < current_contract_expiration]
    return Lumber_expiry, current_contract, current_contract_expiration







In [36]:
Lumber_expiry, current_contract, current_contract_expiration, next_contract, next_contract_expiration = get_full_expiry_df(month_seq)
Lumber_expiry.drop('Month_Start', axis=1, inplace=True)
Lumber_expiry.drop('Expiry', axis=1, inplace=True)
Lumber_expiry_shifted = Lumber_expiry
Lumber_expiry_shifted = Lumber_expiry_shifted[Lumber_expiry_shifted.index <= next_contract_expiration]
Lumber_expiry_shifted = Lumber_expiry_shifted.shift(month_seq-1).dropna()

NameError: name 'month_seq' is not defined

In [ ]:
Lumber_expiry_shifted

In [ ]:
Lumber_expiry_shifted = get_expiry_dfs(month_seq=2)


In [ ]:
Lumber_expiry_shifted[10:]

In [ ]:
month_seq = 2
Lumber_expiry_shifted = None
Lumber_expiry, current_contract, current_contract_expiration, next_contract, next_contract_expiration = get_full_expiry_df()
Lumber_expiry.drop('Month_Start', axis=1, inplace=True)
Lumber_expiry.drop('Expiry', axis=1, inplace=True)
Lumber_expiry_shifted = Lumber_expiry
Lumber_expiry_shifted = Lumber_expiry_shifted[Lumber_expiry_shifted.index <= next_contract_expiration]
Lumber_expiry_shifted = Lumber_expiry_shifted.shift(month_seq-1).dropna()
Lumber_expiry_shifted

In [ ]:
month_seq = 1
Lumber_expiry, current_contract, current_contract_expiration = get_full_expiry_df()
Lumber_expiry.drop('Month_Start', axis=1, inplace=True)
Lumber_expiry.drop('Expiry', axis=1, inplace=True)
Lumber_expiry_shifted = Lumber_expiry.shift(month_seq-1)
Lumber_expiry_shifted = Lumber_expiry_shifted[Lumber_expiry_shifted.index < current_contract_expiration]
Lumber_expiry_shifted


In [ ]:
# Following 2 not required - Change wherever they are used to get_expiry_dfs() and use month_seq
def get_expiries_dfs():
    Lumber_expiry, current_contract, current_contract_expiration = get_LB_expiry()
    Lumber_expiry.drop('Month_Start', axis=1, inplace=True)
    Lumber_expiry.drop('Expiry', axis=1, inplace=True)
    Lumber_expiry_shifted = Lumber_expiry.shift(1)

    Lumber_expiry = add_all_days_to_expiry(Lumber_expiry,total_idx,current_contract,current_contract_expiration).sort_index(ascending=False)
    Lumber_expiry_shifted = add_all_days_to_expiry(Lumber_expiry_shifted,total_idx,current_contract,current_contract_expiration)

    for idx in Lumber_expiry.index:
        dates_dt = str(idx.strftime('%Y-%m-%d'))
        ctrct_expiry = str(Lumber_expiry.loc[idx]['expirationDate'].strftime('%Y-%m-%d'))
        Lumber_expiry.loc[idx,'Days_to_expiry'] = get_days_to_expiry(dates_dt, ctrct_expiry) # 1 means expiring today

        try:
            ctrct_expiry_shift = str(Lumber_expiry_shifted.loc[idx]['expirationDate'].strftime('%Y-%m-%d'))
            Lumber_expiry_shifted.loc[idx,'Days_to_expiry'] = get_days_to_expiry(dates_dt, ctrct_expiry_shift) # 1 means expiring today
        except:
            pass
    return Lumber_expiry, Lumber_expiry_shifted

def get_normal_expiry_dfs():
    Lumber_expiry, current_contract, current_contract_expiration = get_LB_expiry()
    Lumber_expiry.drop('Month_Start', axis=1, inplace=True)
    Lumber_expiry.drop('Expiry', axis=1, inplace=True)

    Lumber_expiry = add_all_days_to_expiry(Lumber_expiry,total_idx,current_contract,current_contract_expiration).sort_index(ascending=False)
    
    for idx in Lumber_expiry.index:
        dates_dt = str(idx.strftime('%Y-%m-%d'))
        ctrct_expiry = str(Lumber_expiry.loc[idx]['expirationDate'].strftime('%Y-%m-%d'))
        Lumber_expiry.loc[idx,'Days_to_expiry'] = get_days_to_expiry(dates_dt, ctrct_expiry) # 1 means expiring today

    return Lumber_expiry

In [ ]:
L1 = get_normal_expiry_dfs()
L2 = get_expiry_dfs(month_seq=1)

In [ ]:
(L1 == L2).all().all()

In [ ]:
def get_adjusted_df():

    Lumber = get_lumber_db_xl() 
    Lumber_expiry, _ , _ = get_LB_expiry()
    Lumber_expiry_normal = get_expiry_dfs(month_seq=1)
    Lumber_expiry_shifted = get_expiry_dfs(month_seq=2)
    Lumber_Adjusted_Prices = Lumber[1].copy(deep=True)
    new_col_name = 'MnthStart_Adj_Close'
    Lumber_Adjusted_Prices[new_col_name] = np.log(Lumber_Adjusted_Prices.close ) - np.log(Lumber_Adjusted_Prices.close.shift(-1))            

    for expiry_dt in Lumber_expiry.index:
        try:
            for dt in pd.date_range(start=Lumber_expiry.loc[expiry_dt,'Month_Start'], end=Lumber_expiry.loc[expiry_dt,'Expiry']):
                dt = dt.strftime('%Y-%m-%d')
            
                if dt in Lumber_Adjusted_Prices.index:
                    Lumber_Adjusted_Prices.loc[dt] = Lumber[2].loc[dt]
                    # Lumber_Adjusted_Prices.loc[dt,Lumber_Adjusted_Prices.columns.drop(['Days_to_expiry','symbol'])] = Lumber[2].loc[dt]

        except:
            pass

    Lumber_Adjusted_Prices[new_col_name] = np.log(Lumber_Adjusted_Prices.close) - np.log(Lumber_Adjusted_Prices.close.shift(-1))

    for dt in Lumber_expiry.Month_Start.values:
        try:
            if (pd.to_datetime(dt) >= Lumber_Adjusted_Prices.index[-1]):
                dt_closest_above = (Lumber_Adjusted_Prices[Lumber_Adjusted_Prices.index >= dt].index[-1])
                dt_closest_below = (Lumber_Adjusted_Prices[Lumber_Adjusted_Prices.index < dt].index[0])
                # dt_expiry
                print(dt,dt_closest_above, dt_closest_below)
                Lumber_Adjusted_Prices.loc[dt_closest_above,new_col_name] = np.log(Lumber_Adjusted_Prices.loc[dt_closest_above,'close']) - np.log(Lumber[2].loc[dt_closest_below,'close'])
        except:
            pass

    Lumber_Adjusted_Prices['Days_to_expiry'] = Lumber_expiry_normal['Days_to_expiry']
    Lumber_Adjusted_Prices['symbol'] = Lumber_expiry_normal['symbol']

    for expiry_dt in Lumber_expiry.index:
        try:
            for dt in pd.date_range(start=Lumber_expiry.loc[expiry_dt,'Month_Start'], end=Lumber_expiry.loc[expiry_dt,'Expiry']):
                dt = dt.strftime('%Y-%m-%d')
            
                if dt in Lumber_Adjusted_Prices.index:
                    Lumber_Adjusted_Prices.loc[dt,'Days_to_expiry'] = Lumber_expiry_shifted.loc[dt,'Days_to_expiry']
                    Lumber_Adjusted_Prices.loc[dt,'symbol'] = Lumber_expiry_shifted.loc[dt,'symbol']
        except:
            pass
    return Lumber_Adjusted_Prices



In [ ]:
Lumber_Adjusted_Prices = get_adjusted_df()
Lumber_Adjusted_Prices

In [ ]:
Lumber_Adjusted_Prices

In [ ]:
Lumber_Adjusted_Prices.MnthStart_Adj_Close.max(), Lumber_Adjusted_Prices.MnthStart_Adj_Close.min()

In [ ]:
Lumber_Adjusted_Prices